In [6]:
import pandas as pd
import numpy as np
#from konlpy.tag import Okt
from khaiii import KhaiiiApi
import gensim
import pyLDAvis.gensim
import numpy as np
import logging
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import time
from pprint import pprint

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
data = pd.read_csv("./modi_data/kor_full.csv") ############
#data = data.drop(['Unnamed: 0'], axis=1)
data.head()

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,year,kor_full
0,137,2002,"탐색 알고리즘 교육을 위한 S/W 컴포넌트의 개발 탐색알고리즘교육, 컴퓨터프로그래밍..."
1,147,2002,프로젝트 학습을 기반으로 하는 ICT 활용 수업 모형의 개발 및 적용 ICT 활용교...
2,148,2002,"분산 컴퓨팅 환경에서의 웹 교육 컴포넌트 개발과정 모델링 웹기반 교육, 컴포넌트 개..."
3,149,2002,효과적인 아동용 WBI를 위한 한글 타이포그래피의 가해성 분석과 활용 타이포 그래피...
4,150,2002,"교수 · 학습 디지털 컨텐트 통합메타데이터 및 개체-관계 모델 설계 교수.학습, 컨..."


In [8]:
dfWordList = pd.read_excel("./word_cor.xlsx")
#print(dfWordList)

dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
#print(dfWordMod)

seriesDelete = dfWordDel["원본"]
stopword = []
for word in seriesDelete.values:
    stopword.append(word)
#print(stopword)

seriesModify = dfWordMod["원본"]
modiword = []
for word in seriesModify.values:
    modiword.append(word)
#print(len(modiword))

seriesModify2 = dfWordMod["수정"]
modiword2 = []
for word in seriesModify2.values:
    modiword2.append(word)
#print(len(modiword2))

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
api = KhaiiiApi()
def khaiiiTokenizer(raw, stopword=stopword, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    for word in api.analyze(raw): #raw data
        for morph in word.morphs:
            if len(word) > 1 and morph in pos and word not in stopword: 
                if morph == 'SL':
                    word = word.lower()
                if word in modiword:
                    word = modiword2[modiword.index(word)]
                list.append(word)        
    return list

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
tokenized = data["kor_full"].apply(lambda row: khaiiiTokenizer(row))
#print(tokenized)
tokenized.to_excel("./final_data/1119token_full.xls") ##############
print("========= tokenization completed =========")

/Users/uni613/Library/Python/3.8/lib/python/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: cannot unpack non-iterable KhaiiiWord object

In [ ]:
#lda
id2word = gensim.corpora.Dictionary(tokenized)

In [ ]:
wordlist = []
for i in range(len(id2word)):
    #print(id2word[i])
    wordlist.append(id2word[i])
#print(wordlist)
seriesWordlist = pd.Series(wordlist)
seriesWordlist.to_excel("./final_data/1119wordlist_full.xls") #################

corpus=[id2word.doc2bow(text) for text in tokenized]
#print("id2word for each document : ", corpus)
print("# words in total : ", len(id2word))
print("# documents : ", len(corpus))